In [ ]:
import sys
sys.path.append('/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/')
from astropy.io import ascii
import numpy as np
import pandas as pd
import matplotlib
import csv
import scipy
import matplotlib.pyplot as plt
from seaborn import histplot
import random
import copy
from os import path, makedirs
from astropy.coordinates import SkyCoord
from astropy.table import Table
import astropy.units as u
from ctapipe.containers import MuonEfficiencyContainer
from ctapipe.coordinates import CameraFrame, TelescopeFrame
from ctapipe.io import EventSource, EventSeeker
from ctapipe.core import TelescopeComponent
from ctapipe.core.traits import (
    List,
    Int,
    FloatTelescopeParameter,
    TelescopeParameter,
    Unicode,
)
from ctapipe.image.cleaning import tailcuts_clean
from ctapipe.image.muon import MuonRingFitter, MuonIntensityFitter
from traitlets.config import Config
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from ctapipe.calib import CameraCalibrator
from traitlets.config.loader import Config, FileConfigLoader, JSONFileConfigLoader
import pathlib
from calibpipe.throughput import MuonCalibrator
from traitlets.config import Config
from astropy.time import Time
from astropy.coordinates import EarthLocation,SkyCoord, AltAz 
from traitlets.config import Config
import time
from functools import reduce





In [ ]:
event_container = {}
counter = 0
for k in range(1,41):
    filename = f'/Users/vdk/sim_run_folder/output_prod_v12/corsika_theta_28.4_az_182.75_run{k}.simtel.gz'
    source = EventSource(filename)
    event_iterator = iter(source)
    keks = k*1000
    for i,j in enumerate(event_iterator):
        counter += 1
        if len(j.trigger.tels_with_trigger) > 1:
            event_container[keks+i] = j

In [ ]:
high_energy_events = {}
for i in event_container:
    if event_container[i].simulation.shower.energy.to_value() > 10:
        print(event_container[i].simulation.shower.energy)
        high_energy_events[i] = event_container[i]

In [ ]:
high_energies = []
for i in high_energy_events:
    high_energies.append(high_energy_events[i].simulation.shower.energy.to_value())

In [ ]:
x,y,_ = plt.hist(high_energies, bins = 10, alpha = 0.5, log = True)
plt.grid(alpha = 0.5)

In [ ]:
x,y,_ = plt.hist(high_energies, bins = 12, alpha = 0.5, log = False)
plt.grid(alpha = 0.5)

In [ ]:
high_energy_events[1018].trigger.tel[3].trigger_pixels

In [ ]:
event = high_energy_events[1018]
image_processor = ImageProcessor(source.subarray)
muon_processor = MuonProcessor(source.subarray)
calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum", subarray = source.subarray)
calib(event)
image_processor(event)
muon_processor(event)

In [ ]:
high_energy_events[1018]

In [ ]:
print(high_energy_events[1018].simulation)

In [ ]:
len(high_energy_events)

In [ ]:
flatten_impact = sum(impact, [])

In [ ]:
impact = np.array(impact)

In [ ]:
impact = []
for i in high_energy_events:
    impact_points = []
    for tel in high_energy_events[i].trigger.tels_with_trigger:
        impact_points.append(high_energy_events[i].simulation.tel[tel].impact.distance.to_value())
    impact.append(impact_points)

In [ ]:
impact_xyz = []
impact_min = []
impact_max = []
for i in impact:
    impact_min.append(min(i))
    impact_max.append(max(i))    
    if len(i) > 2:
        impact_xyz.append(i)

In [ ]:
plt.hist(impact_min, alpha = 0.5, label = 'min impact distance for event')
plt.hist(impact_max, alpha = 0.5, label = 'maximum impact distance for event')
#plt.hist(flatten_impact, histtype = 'step', label = 'impact distance for every telescope')
plt.legend()
plt.grid(alpha = 0.5)
plt.xlabel('impact distance from telescope in meters')
plt.ylabel('events')


In [ ]:
plt.hist(flatten_impact, histtype = 'step', label = 'impact distance for every telescope')
plt.legend()
plt.grid(alpha = 0.5)
plt.xlabel('impact distance from telescope in meters')
plt.ylabel('events')



In [ ]:
def number_events(E0, E1, k = -2.68, A = 8040987.083198826): #A = 1000100.01):
    return A * (E1**(k+1)-E0**(k+1))/(k+1)

energies = np.logspace(1,5,45)

events = number_events(energies[:-1], energies[1:])
plt.step(0.5*(energies[:-1] + energies[1:]), events)
plt.plot([10000 for i in range(100000)], range(100000))
plt.yscale('symlog')
plt.xscale('symlog')
plt.xlabel("Energy [GeV]")
plt.ylabel("Number of showers")
plt.grid(alpha = 0.5)
#plt.annotate("Power law with index = -2", xy = (1000,210), xytext = (100,10),arrowprops=dict(facecolor='black', shrink=0.05, width = 1, headwidth = 5))
plt.annotate("Power law with index = -2.68", xy = (150,100), xytext = (10,5),arrowprops=dict(facecolor='black', shrink=0.05, width = 1, headwidth = 5))

events

In [ ]:
energies[energies >= 10000]

In [ ]:
energies[len(energies)-12:]

In [ ]:
sum(events[len(events)-12:])

In [ ]:
(sum(events[len(events)-12:])/sum(events))

In [ ]:
sum(events)

In [ ]:
events

In [ ]:
A = 100000 / ((100000**(-1.68) - 10**(-1.68))/-1.68)
A

In [ ]:
0.36*10500

In [ ]:
1215/18000

In [ ]:
0.0011329600354456106*18000

In [ ]:
1215/4e7*100

In [ ]:
(1215/4e7)/(sum(events[len(events)-12:])/sum(events))

In [ ]:
print(event.pointing.tel[1])

In [ ]:
print(np.rad2deg(event.pointing.tel[1].azimuth.to_value()))
print(np.rad2deg(event.pointing.tel[1].altitude.to_value()))

In [ ]:
event.pointing.tel[1].azimuth.to_value()